In [12]:
import traceback

wml_credentials = {
    "username":"admin",
    "password":"password",
    "instance_id" : "openshift",
    "url": "https://cpd-cpd-402.apps.ocp48nfsqacluster2.cp.fyre.ibm.com", # PUT YOUR CPD URL HERE
    "version": "4.0"
}

from ibm_watson_machine_learning import APIClient
client = APIClient(wml_credentials)

In [13]:
# Set pretrained file directory
wml_artifacts_dir = "/userfs/assets/data_asset/"

In [14]:
# Set default space or project
#client.set.default_space('SPACE ID')
client.set.default_project('19fb73a1-d33e-46f8-a5a5-b9a6e9418d5e/')

'SUCCESS'

In [15]:
model_list = [
    {'runtime': 'default_py3.8',
  'type': 'scikit-learn_0.23',
  'name': 'scikit-learn_0.23_model',
  'file': 'scikit-23-boston-pretrained.tar.gz',
  'payload' : [{"values":[[0.00632, 18.0, 2.31, 0.0, 0.538, 6.575, 65.2, 4.09, 1.0, 296.0, 15.3, 396.9, 4.98], [0.02731, 0.0, 7.07, 0.0, 0.469, 6.421, 78.9, 4.9671, 2.0, 242.0, 17.8, 396.9, 9.14], [0.02729, 0.0, 7.07, 0.0, 0.469, 7.185, 61.1, 4.9671, 2.0, 242.0, 17.8, 392.83, 4.03], [0.03237, 0.0, 2.18, 0.0, 0.458, 6.998, 45.8, 6.0622, 3.0, 222.0, 18.7, 394.63, 2.94], [0.06905, 0.0, 2.18, 0.0, 0.458, 7.147, 54.2, 6.0622, 3.0, 222.0, 18.7, 396.9, 5.33]]}]
},
{'runtime': 'default_py3.7_opence',
  'type': 'tensorflow_2.4',
  'name': 'pretrained-tf-2_4-opence-iris-model',
  'file': 'iris_keras_sequential_2.4.tar.gz',
  'payload' : [{
	"values": [
[5.7, 2.9, 4.2, 1.3]
       ]
}]},
{'runtime': 'tensorflow_2.4-py3.7',
  'type': 'tensorflow_2.4',
  'name': 'pretrained-tf-2_4-non-default-iris-model',
  'file': 'iris_keras_sequential_2.4.tar.gz',
  'payload' : [{
	"values": [
[5.7, 2.9, 4.2, 1.3]
       ]
}]}
]

In [16]:
# CREATE MODELS ONLY

from random import randint
# Create models in the space
# Initialize name, detail, and guid lists
model_guid_list = []
model_name_list = []
model_detail_dict = {}

for model in model_list:
    # Initialize cleanup lists
    asset_cleanup = []
    deployment_cleanup = []
    print("\nCreating model...\n")
    
    model_name = model['name']
    
    try:
        # Get the software specification for the model if available
        software_spec_uid = client.software_specifications.get_uid_by_name(model['runtime'])
    except BaseException:
        raise Exception ("Failed retrieve software spec uid for " + model_name + ". Exception: " + traceback.format_exc())
    
    try:
        # Save the model
        if software_spec_uid == "Not Found":
            print("Software spec for " + model['runtime'] + " not found. Creating model using runtime")
            
            unique_model_name = model['name']
            
            meta_props={
             client.repository.ModelMetaNames.NAME: unique_model_name,
             client.repository.ModelMetaNames.RUNTIME_UID: model['runtime'],
             client.repository.ModelMetaNames.TYPE: model['type']
            }

            created_model_details = client.repository.store_model(wml_artifacts_dir+model["file"], meta_props)
        else:
            print("Software specification for " + model['runtime'] + " found. Creating model with software specification")
            
            unique_model_name = model['name']
                                                    
            meta_props={
             client.repository.ModelMetaNames.NAME: unique_model_name,
             client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
             client.repository.ModelMetaNames.DESCRIPTION: "SVT Model with Software Spec",
             client.repository.ModelMetaNames.TYPE: model['type']
            }

            created_model_details = client.repository.store_model(wml_artifacts_dir+model["file"], meta_props)
    except BaseException:
        raise Exception ("Failed to create " + unique_model_name + " and save to Repo. Exception: " + traceback.format_exc())
        
    #model_name = created_model_details['entity']['name']
    
    # Add details to a list for later access if neccessary
    model_detail_dict.update({unique_model_name : created_model_details})
    
    # Add to name list for display later
    model_name_list.append(unique_model_name)
    print(unique_model_name + " created...\n")
    
    try:
        # Get model uid
        model_uid = client.repository.get_model_uid(created_model_details)
    except BaseException:
        raise Exception ("Failed to retrieve UID for " + model_name + ". Exception: " + traceback.format_exc())
        
    model_guid_list.append(model_uid)
    
print("\nCompleted model creation.\n")


Creating model...



Exception: Failed retrieve software spec uid for scikit-learn_0.23_model. Exception: Traceback (most recent call last):
  File "<ipython-input-16-cce3ec022e00>", line 20, in <module>
    software_spec_uid = client.software_specifications.get_uid_by_name(model['runtime'])
  File "/opt/ibm/conda/miniconda/lib/python/site-packages/ibm_watson_machine_learning/sw_spec.py", line 366, in get_uid_by_name
    total_values = self._handle_response(200, u'list assets', response)["total_results"]
  File "/opt/ibm/conda/miniconda/lib/python/site-packages/ibm_watson_machine_learning/wml_resource.py", line 73, in _handle_response
    raise ApiRequestFailure(u'Failure during {}.'.format(operationName), response)
ibm_watson_machine_learning.wml_client_error.ApiRequestFailure: Failure during list assets. (GET https://cpd-cpd-402.apps.ocp48nfsqacluster2.cp.fyre.ibm.com/v2/software_specifications?version=2021-06-24&project_id=19fb73a1-d33e-46f8-a5a5-b9a6e9418d5e%2F&userfs=true&name=default_py3.8)
Status co